In [1]:
import json
import os
import PIL
import PIL.Image
from IPython.display import display
from PIL import Image
import time

import keras

from keras_preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np



# Import all the Keras machinery we need
from keras import applications
from keras import optimizers
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import metrics
from keras.utils import to_categorical



from gen_data import generate_chip

Using TensorFlow backend.


(255, 150, 4)


In [2]:
labels = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789/'
label_d = {}
for index, k in enumerate(labels):
    label_d[k] = index
#print(label_d)

In [3]:
def get_input():

    chip, coords, letters, letter_size = generate_chip()
    
    return chip, coords, letters, letter_size

def get_output(letter, label_file=None):

    # img_id = path.split('/')[-1].split('.')[0]
    # img_id = np.int64(img_id)
    # labels = label_file.loc[img_id].values

    return(label_d[letter])

def preprocess_input(chip, coord, letter_size):

    image = chip.crop((coord[0], coord[1], coord[0]+letter_size[0], coord[1]+letter_size[1]))
    im_resized = image.resize((224,224))
    im_final  = im_resized.convert('RGB')
    im_final = np.array(im_final)
    #print(im_final.shape)
    #print(im_resized.mode)

    return(im_final)

def image_generator(batch_size):

    while True:
        # Select files (paths/indices) for the batch
        # batch_paths = np.random.choice(a = files,
        #                                  size = batch_size)
        batch_input = []
        batch_output = []

        # Read in each input, perform preprocessing and get labels
        for i in range(batch_size):
            chip, coords, letters, letter_size = get_input()
            #print(letters)
            for i in range(4):#len(letters)):
                output = get_output(letters[i])
                input = preprocess_input(chip, coords[i], letter_size)
                batch_input += [ input ]
                batch_output += [ output ]

        # Return a tuple of (input,output) to feed the network
        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
    
        print('BATCH GENERATED')
        
        
        yield( batch_x, to_categorical(batch_y) )
        
def image_generator2(batch_size):

    while True:
        # Select files (paths/indices) for the batch
        # batch_paths = np.random.choice(a = files,
        #                                  size = batch_size)

    
              
        
        chip, coords, letters, letter_size = get_input()
        step = len(letters) // batch_size
        print(step)

        for i in range(step):
            batch_input = []
            batch_output = []

            # Return a tuple of (input,output) to feed the network


            for i in range(len(letters)):
                output = get_output(letters[i])
                input = preprocess_input(chip, coords[i], letter_size)
                batch_input += [ input ]
                batch_output += [ output ]
            batch_x = np.array( batch_input )
            batch_y = np.array( batch_output )   
            print('BATCH GENERATED') 
            #print('batch_x',batch_x)
            yield( batch_x, to_categorical(batch_y) )

In [5]:
n_train_samples = 200
n_val_samples =  1
batch_size = 1
num_classes = 37


train_generator = image_generator2(batch_size)

validation_generator = image_generator2(batch_size)



In [8]:
'''
# Build a simple CONVNET
# Like in https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

if K.image_data_format() == 'channels_first':
    input_shape = (3, 224, 224)
else:
    input_shape = (224, 224, 3)

cnnmodel = Sequential()
cnnmodel.add(Conv2D(32, (3, 3), input_shape=input_shape))
cnnmodel.add(Activation('relu'))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))


cnnmodel.add(Conv2D(64, (3, 3)))
cnnmodel.add(Activation('relu'))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))




cnnmodel.add(Flatten())
cnnmodel.add(Dense(64))
cnnmodel.add(Activation('relu'))


cnnmodel.add(Dropout(0.5))
cnnmodel.add(Dense(num_classes))
cnnmodel.add(Activation('softmax'))



cnnmodel.compile(loss='categorical_crossentropy',
                optimizer=optimizers.Adadelta(),
                metrics=['accuracy'])

#K.clear_session()
'''

"\n# Build a simple CONVNET\n# Like in https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html\nfrom keras.models import Sequential\nfrom keras.layers import Conv2D, MaxPooling2D\nfrom keras.layers import Activation, Dropout, Flatten, Dense\n\nif K.image_data_format() == 'channels_first':\n    input_shape = (3, 224, 224)\nelse:\n    input_shape = (224, 224, 3)\n\ncnnmodel = Sequential()\ncnnmodel.add(Conv2D(32, (3, 3), input_shape=input_shape))\ncnnmodel.add(Activation('relu'))\ncnnmodel.add(MaxPooling2D(pool_size=(2, 2)))\n\n\ncnnmodel.add(Conv2D(64, (3, 3)))\ncnnmodel.add(Activation('relu'))\ncnnmodel.add(MaxPooling2D(pool_size=(2, 2)))\n\n\n\n\ncnnmodel.add(Flatten())\ncnnmodel.add(Dense(64))\ncnnmodel.add(Activation('relu'))\n\n\ncnnmodel.add(Dropout(0.5))\ncnnmodel.add(Dense(num_classes))\ncnnmodel.add(Activation('softmax'))\n\n\n\ncnnmodel.compile(loss='categorical_crossentropy',\n                optimizer=optimizers.Adadelta(),\n          

In [6]:
from keras.layers import Activation, Dropout, Flatten, Dense
model_name = 'gros_resnet50'

# number of classes

# input image shape
input_shape = (224, 224, 3)

# load base model
base_model = ResNet50(weights='imagenet', include_top=False,
                  input_shape=input_shape)

# append classification layer
x = base_model.output
x = Flatten()(x)
final_output = Dense(num_classes, activation='softmax', name='fc360')(x)

# create the new model
model = Model(inputs=base_model.input, outputs=final_output)

model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.Adam(lr = 1e-4),
                metrics=['accuracy'])

# set the first n layers (up to the last conv block)
# to non-trainable (weights will not be updated)
#for layer in model.layers[:10]:
#    layer.trainable = False
    
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/home/abert/.virtualenvs/test/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [9]:

from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from time import time

# callbacks
monitor = 'loss'
checkpointer = ModelCheckpoint(
    filepath=os.path.join('./', model_name + '.hdf5'),
    monitor=monitor,
    save_best_only=True
)
reduce_lr = ReduceLROnPlateau(monitor=monitor, factor = 0.3, patience=3)
early_stopping = EarlyStopping(monitor=monitor, patience=5)
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

history = model.fit_generator(
    train_generator,
    steps_per_epoch=n_train_samples // batch_size,
    epochs=2,
    #validation_data=validation_generator,
    #validation_steps=n_val_samples // batch_size
    callbacks=[checkpointer, tensorboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
18
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
BATCH GENERATED
  7/200 [>.............................] - ETA: 6:46 - loss: 0.6932 - acc: 0.865118
BATCH GENERATED
 25/200 [==>...........................] - ETA: 3:39 - loss: 1.7364 - acc: 0.862218
BATCH GENERATED
 43/200 [=====>........................] - ETA: 2:55 - loss: 3.4044 - acc: 0.771318
BATCH GENERATED
 61/200 [========>.....................] - ETA: 2:27 - loss: 4.2672 - acc: 0.722218
BATCH GENERATED
 79/200 [==========>...................] - ETA: 2:04 - loss: 4.4515 - acc: 0.712418
BATCH GENERATED


 97/200 [=============>................] - ETA: 1:44 - loss: 4.2960 - acc: 0.723918
BATCH GENERATED
115/200 [================>.............] - ETA: 1:24 - loss: 4.0362 - acc: 0.740618
BATCH GENERATED
133/200 [==================>...........] - ETA: 1:06 - loss: 3.7777 - acc: 0.757318
BATCH GENERATED
151/200 [=====================>........] - ETA: 47s - loss: 3.5194 - acc: 0.773018
BATCH GENERATED
163/200 [=======================>......] - ETA: 36s - loss: 3.3341 - acc: 0.7843BATCH GENERATED


169/200 [========================>.....] - ETA: 30s - loss: 3.2479 - acc: 0.789618
BATCH GENERATED
187/200 [===========================>..] - ETA: 12s - loss: 3.0427 - acc: 0.802718
BATCH GENERATED
200/200 [==============================] - 194s 971ms/step - loss: 2.9166 - acc: 0.8106
Epoch 2/2
BATCH GENERATED
  5/200 [..............................] - ETA: 3:09 - loss: 1.8025 - acc: 0.888918
BATCH GENERATED
 23/200 [==>...........................] - ETA: 2:49 - loss: 2.4898 - acc: 0.840618
BATCH GENERATED
 41/200 [=====>........................] - ETA: 2:30 - loss: 2.9710 - acc: 0.813018
BATCH GENERATED
 47/200 [======>.......................] - ETA: 2:24 - loss: 3.0490 - acc: 0.8085BATCH GENERATED


 59/200 [=======>......................] - ETA: 2:12 - loss: 2.9474 - acc: 0.814518
BATCH GENERATED
 77/200 [==========>...................] - ETA: 1:55 - loss: 2.6344 - acc: 0.833318
BATCH GENERATED
 95/200 [=============>................] - ETA: 1:37 - loss: 2.4437 - acc: 0.845618
BATCH GENERATED
113/200 [===============>..............] - ETA: 1:20 - loss: 2.4296 - acc: 0.846118
BATCH GENERATED
131/200 [==================>...........] - ETA: 1:04 - loss: 2.3935 - acc: 0.847818
BATCH GENERATED


149/200 [=====================>........] - ETA: 47s - loss: 2.3872 - acc: 0.848618
BATCH GENERATED
167/200 [========================>.....] - ETA: 30s - loss: 2.3988 - acc: 0.848018
BATCH GENERATED
185/200 [==========================>...] - ETA: 13s - loss: 2.3804 - acc: 0.849218
BATCH GENERATED
200/200 [==============================] - 186s 932ms/step - loss: 2.2049 - acc: 0.8603


In [8]:
print([1]+[2])

[1, 2]
